In [3]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from matplotlib import style
style.use('ggplot')
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
from wordcloud import WordCloud
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [5]:
train = pd.read_csv('/content/Google play store Review.csv')
train.head()

,content,score
0,I cannot open the app anymore,1
1,I have been begging for a refund from this app...,1
2,Very costly for the premium version (approx In...,1
3,"Used to keep me organized, but all the 1010 UP...",1
4,Dan Birthday Oct 18,1


In [6]:
def remove_pattern(text,pattern):

    # re.findall() finds the pattern i.e @user and puts it in a list for further task
    r = re.findall(pattern,text)

    # re.sub() removes @user from the sentences in the dataset
    for i in r:
        text = re.sub(i,"",text)

    return text

In [8]:
train['Tidy_Reviews'] = np.vectorize(remove_pattern)(train['content'], "@[\w]*")
train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")
train['Tidy_Reviews'] = train['Tidy_Reviews'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
train.head()

<ipython-input-8-db8fca9423f4>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  train['Tidy_Reviews'] = train['Tidy_Reviews'].str.replace("[^a-zA-Z#]", " ")


,content,score,Tidy_Reviews
0,I cannot open the app anymore,1,cannot open anymore
1,I have been begging for a refund from this app...,1,have been begging refund from this over month ...
2,Very costly for the premium version (approx In...,1,Very costly premium version approx Indian Rupe...
3,"Used to keep me organized, but all the 1010 UP...",1,Used keep organized UPDATES have made mess thi...
4,Dan Birthday Oct 18,1,Birthday


In [9]:
tokenized_review = train['Tidy_Reviews'].apply(lambda x: x.split())
tokenized_review.head()

0                              [cannot, open, anymore]
1    [have, been, begging, refund, from, this, over...
2    [Very, costly, premium, version, approx, India...
3    [Used, keep, organized, UPDATES, have, made, m...
4                                           [Birthday]
Name: Tidy_Reviews, dtype: object

In [10]:
from nltk import PorterStemmer
ps = PorterStemmer()
tokenized_review = tokenized_review.apply(lambda x: [ps.stem(i) for i in x])
tokenized_review.head()

0                               [cannot, open, anymor]
1    [have, been, beg, refund, from, thi, over, mon...
2    [veri, costli, premium, version, approx, india...
3    [use, keep, organ, updat, have, made, mess, th...
4                                           [birthday]
Name: Tidy_Reviews, dtype: object

In [11]:
for i in range(len(tokenized_review)):
    tokenized_review[i] = ' '.join(tokenized_review[i])

train['Tidy_Reviews'] = tokenized_review
train.head()

,content,score,Tidy_Reviews
0,I cannot open the app anymore,1,cannot open anymor
1,I have been begging for a refund from this app...,1,have been beg refund from thi over month nobod...
2,Very costly for the premium version (approx In...,1,veri costli premium version approx indian rupe...
3,"Used to keep me organized, but all the 1010 UP...",1,use keep organ updat have made mess thing cudn...
4,Dan Birthday Oct 18,1,birthday


In [12]:
from sklearn.feature_extraction.text import CountVectorizer

bow_vectorizer = CountVectorizer(max_df=0.90, min_df=2, max_features=1000, stop_words='english')

# bag-of-words feature matrix
bow = bow_vectorizer.fit_transform(train['Tidy_Reviews'])

df_bow = pd.DataFrame(bow.todense())

df_bow

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12481,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12482,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
12483,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(max_df=0.90, min_df=2,max_features=1000,stop_words='english')
tfidf=tfidf.fit_transform(train['Tidy_Reviews'])
df_tfidf = pd.DataFrame(tfidf.todense())
df_tfidf

,0,1,2,3,4,5,6,7,8,9,...,990,991,992,993,994,995,996,997,998,999
0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
1,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.234293,0.0,0.0,0.0
3,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
4,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12480,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12481,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12482,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0
12483,0.0,0.0,0.0,0.468929,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0


In [14]:
train_bow = bow[:12485]

train_bow.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [15]:
train_tfidf= tfidf[:12485]

train_tfidf.todense()

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [16]:
from sklearn.model_selection import train_test_split

In [17]:
x_train_bow,x_valid_bow,y_train_bow,y_valid_bow = train_test_split(train_bow,train['score'],test_size=0.3,random_state=2)
x_train_tfidf,x_valid_tfidf,y_train_tfidf,y_valid_tfidf = train_test_split(train_tfidf,train['score'],test_size=0.3,random_state=17)

In [18]:
pip install keras.preprocessing

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 682.4 kB/s eta 0:00:00


In [19]:
from keras.preprocessing.text import one_hot, Tokenizer
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, GlobalMaxPooling1D, Embedding, Conv1D, LSTM
from sklearn.model_selection import train_test_split
from keras_preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from numpy import array
import tensorflow as tf

from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D

In [20]:
x_train_bow = x_train_bow.toarray()
x_valid_bow = x_valid_bow.toarray()

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_bow.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))  # Additional Convolutional Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))  # Additional Dense Layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_bow, y_train_bow, batch_size=5, epochs=15)


Epoch 1/15
1748/1748 [==============================] - 70s 39ms/step - loss: 0.5325 - accuracy: 0.7285
Epoch 2/15
1748/1748 [==============================] - 48s 28ms/step - loss: 0.4371 - accuracy: 0.7916
Epoch 3/15
1748/1748 [==============================] - 51s 29ms/step - loss: 0.3714 - accuracy: 0.8316
Epoch 4/15
1748/1748 [==============================] - 49s 28ms/step - loss: 0.2845 - accuracy: 0.8794
Epoch 5/15
1748/1748 [==============================] - 47s 27ms/step - loss: 0.2003 - accuracy: 0.9199
Epoch 6/15
1748/1748 [==============================] - 47s 27ms/step - loss: 0.1406 - accuracy: 0.9463
Epoch 7/15
1748/1748 [==============================] - 47s 27ms/step - loss: 0.1153 - accuracy: 0.9567
Epoch 8/15
1748/1748 [==============================] - 64s 37ms/step - loss: 0.0929 - accuracy: 0.9640
Epoch 9/15
1748/1748 [==============================] - 48s 27ms/step - loss: 0.0877 - accuracy: 0.9660
Epoch 10/15
1748/1748 [==============================] - 47s 27m

In [22]:
score = model.evaluate(x_valid_bow, y_valid_bow, verbose=1)

118/118 [==============================] - 3s 21ms/step - loss: 1.6201 - accuracy: 0.7643


In [23]:
x_train_tfidf = x_train_tfidf.toarray()
x_valid_tfidf = x_valid_tfidf.toarray()

In [24]:
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten

model = Sequential()
model.add(Conv1D(filters=32, kernel_size=3, activation='relu', input_shape=(x_train_tfidf.shape[1], 1)))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=64, kernel_size=3, activation='relu'))
model.add(MaxPooling1D(pool_size=2))
model.add(Conv1D(filters=128, kernel_size=3, activation='relu'))  # Additional Convolutional Layer
model.add(MaxPooling1D(pool_size=2))
model.add(Flatten())
model.add(Dense(units=64, activation='relu'))  # Additional Dense Layer
model.add(Dense(units=1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
history = model.fit(x_train_tfidf, y_train_tfidf, batch_size=5, epochs=15)


Epoch 1/15
1748/1748 [==============================] - 51s 28ms/step - loss: 0.5339 - accuracy: 0.7211
Epoch 2/15
1748/1748 [==============================] - 46s 27ms/step - loss: 0.4530 - accuracy: 0.7805
Epoch 3/15
1748/1748 [==============================] - 47s 27ms/step - loss: 0.3980 - accuracy: 0.8151
Epoch 4/15
1748/1748 [==============================] - 47s 27ms/step - loss: 0.3283 - accuracy: 0.8573
Epoch 5/15
1748/1748 [==============================] - 46s 26ms/step - loss: 0.2384 - accuracy: 0.9079
Epoch 6/15
1748/1748 [==============================] - 46s 26ms/step - loss: 0.1532 - accuracy: 0.9430
Epoch 7/15
1748/1748 [==============================] - 49s 28ms/step - loss: 0.1123 - accuracy: 0.9594
Epoch 8/15
1748/1748 [==============================] - 45s 26ms/step - loss: 0.0904 - accuracy: 0.9662
Epoch 9/15
1748/1748 [==============================] - 46s 26ms/step - loss: 0.0758 - accuracy: 0.9724
Epoch 10/15
1748/1748 [==============================] - 44s 25m

In [25]:
score = model.evaluate(x_valid_tfidf, y_valid_tfidf, verbose=1)

118/118 [==============================] - 3s 21ms/step - loss: 1.5832 - accuracy: 0.7499
